# How to create nice plots? 3Tips👍📊

## **Content**
1. [Data and library loading](#1)
1. [Examples of visualization](#2)
1. [Tip1: Subplot](#3)
1. [Tip2: Style](#4)
1. [Tip3: Color](#5)

## Summary

I introduced tips to create impressed nice plots using following 3 techniques.

- Subplot

- Style

- Color

## Motivation

It's very important to create a compact, compelling and impressive graph. 

Due to the nature of jutyter notebooks, our analysis usually proceeds vertically. Readers might get tired or difficult to understand the relationships between the graphs. 

If we could arrange the graphs in an orderly fashion, the readers might get better understanding for the relationships between the graphs, which may lead to deeper insights. If we can create a impressive and memorable figure, readers may look at the graph more carefully, or they may recall it in other competitions and use the findings to their advantage.

Of course, changing the color and shape of the graph can create misunderstandings and, in some cases, deceive readers. But basically, I think devices that makes the graph look better are a positive thing.

In this notebook,  I'll explain the techniques we need to create nice graphs and its' references.

<a id="1"></a> <br>
# <div class="alert alert-block alert-info">Data and library loading</div>

First, I'll import libraries and dataset.

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from plotly.subplots import make_subplots
import statsmodels.api as sm
import plotly.graph_objs as go
import seaborn as sns

%matplotlib inline

In [ ]:
! ls ../input/nfl-big-data-bowl-2021/

In [ ]:
df_players = pd.read_csv("../input/nfl-big-data-bowl-2021/players.csv")

In [ ]:
df_players.head()

<a id="2"></a> <br>
# <div class="alert alert-block alert-info">Examples of visualization</div>

For example, I'll take a look at the following three perspectives on defensive players

- Count of position

- Defensive player's height

- Defensive player's weight

I'll pull out the data on the defensive players.

In [ ]:
defence = ["DL", "DE", "DT","LB", "OLB", "MLB", "CB", "FS", "SS", "S", "DB"]
df_players = df_players[df_players["position"].isin(defence)].reset_index(drop=True)

First, I count up number of players' position.

In [ ]:
g_position = sns.countplot(df_players["position"])
g_position.set_title("Number of players per position")

Next, I'll see distribution of players' height and weight.

In [ ]:
# Inspired by https://www.kaggle.com/fatihbilgin/nfl-big-data-visualization
df_players["HeightFt"] = df_players["height"].str.split('-', expand=True)[0].astype(int)
df_players["HeightIn"] = df_players["height"].str.split('-', expand=True)[1].fillna(0).astype(int)
df_players["HeightCm"] = df_players["HeightFt"]*30.48 + df_players["HeightIn"]*2.54

df_players["WeightKg"] = df_players["weight"]*0.45359237

df_height = df_players.groupby(['height','HeightFt','HeightIn']).size().reset_index().sort_values(["HeightFt", "HeightIn"])

df_height.columns = ["height","HeightFt","HeightIn","Count"]

In [ ]:
g_height = df_height.loc[:,["height","Count"]].set_index("height").plot(color='green', kind='bar')

g_height.set_xlabel("Height") 
g_height.set_ylabel("Count") 
g_height.get_legend().remove()
g_height.set_title('Player Height (ft-in)')

In [ ]:
g_weight = sns.distplot(df_players["weight"], kde=False, rug=False)

g_weight.set_xlabel("Weight") 
g_weight.set_title('Player Weight (lbs)')

Now, we've looked at the results of the three visualization so far. But the graphs are hard to see because they are in a vertical line. It is difficult to compare between the graphs, and the results of each analysis are not memorable.

In the next section, we'll take advantage of the subplot to solve this problem.

<a id="1"></a> <br>
# <div class="alert alert-block alert-info">Tip1: Subplot</div>

First, let's look at the results!

In [ ]:
#Create figure and Axes. And set title.
fig, axes = plt.subplots(2, 2, figsize=(10,6), gridspec_kw=dict(wspace=0.1, hspace=0.6))
fig.suptitle("Defensive players' information", fontsize=15)

#Too check layout, I'll show text on each Axes.
gs = axes[0, 1].get_gridspec()
axes[0, 0].remove()
axes[1, 0].remove()
#Add gridspec we got
axbig = fig.add_subplot(gs[:, 0])


#Add three plots.
sns.countplot(df_players["position"], ax=axbig)
axbig.set_title("Number of players per position", fontsize=12)

df_height.loc[:,["height","Count"]].set_index("height").plot(kind='bar', ax=axes[0, 1])
axes[0, 1].set_ylabel("Count") 
axes[0, 1].get_legend().remove()
axes[0, 1].set_title('Player Height (ft-in)', fontsize=12)

sns.distplot(df_players["weight"], kde=False, rug=False, ax=axes[1, 1])
axes[1, 1].set_xlabel("Weight") 
axes[1, 1].set_title('Player Weight (lbs)', fontsize=12)

We were able to fit the three graphs into a single diagram. Now you can see the graphs at a time without scrolling through the screen! In addition, by combining them into a single piece, we are able to create a single theme, "Defensive players' information".

The readers must get be more interested in the factor and more likely to consider it than when it was presented in a disjointed fashion.

Now, let's talk about how to make it. We can create subplots by using plt.subplots. For example, to split the screen into a 2x2 screen and arrange the graph, do the following,

In [ ]:
#Create figure and Axes. And set title.
fig, axes = plt.subplots(2, 2, figsize=(10,5))
fig.suptitle('2 row x 2 columns axes with no data')

#Too check layout, I'll show text on each Axes.
for col in range(axes.shape[0]):
    for row in range(axes.shape[1]):
        label = 'Col: {}\nRow: {}'.format(col, row)
        axes[col][row].annotate(label, (0.1, 0.5), xycoords='axes fraction', va='center')

By plt.subplots, the number of vertical and horizontal divisions will be equal. If you want to create non-equal layout subplots, there are some ways, but I'll show you following way.

1. Use get_gridspec() to get the geometry information of the Axes you want to combine, and remove the Axes that are no longer needed with remove().

1. Based on the geometry we just acquired, we will place the new large Axes.

In [ ]:
#Create figure and Axes. And set title.
fig, axes = plt.subplots(2, 2, figsize=(10,5))
fig.suptitle('Bigger 1 row x 2 columns axes with no data')

#Too check layout, I'll show text on each Axes.
for col in range(axes.shape[0]):
    for row in range(axes.shape[1]):
        label = 'Col: {}\nRow: {}'.format(col, row)
        axes[col][row].annotate(label, (0.1, 0.5), xycoords='axes fraction', va='center')

#Get gridspec and remove unwanted Axes objects
gs = axes[0, 1].get_gridspec()
axes[0, 0].remove()
axes[1, 0].remove()

#Add gridspec we got
axbig = fig.add_subplot(gs[:, 0])

#Too check layout, I'll show text on the new big Axes.
axbig.annotate('Big Axes \nGridSpec[:, 0]', (0.1, 0.5),
               xycoords='axes fraction', va='center')

If we can do these two things, it should be enough for now.

If you want ot know more detail, you can see [Customizing Figure Layouts Using GridSpec and Other Functions](https://matplotlib.org/3.3.2/tutorials/intermediate/gridspec.html).

<a id="1"></a> <br>
# <div class="alert alert-block alert-info">Tip2: Style</div>

We could place multiple graphs  in a single image, compactly. But the image's color remain the default and the graph has no personality and I've seen its' design anywhere.

To improve this problem, we can change graphs' style by using plt.style.use(). If we choose style, we can easilly chage graph's color, font and so on.  

We can check about style in [Controlling figure aesthetics](https://seaborn.pydata.org/tutorial/aesthetics.html).

You can see all options by plt.style.available.

In [ ]:
plt.style.available

Let's look . Let's apply all styles to our plot.

In [ ]:
def titanic_subplot(style_available):
    #set style
    plt.style.use(style_available)
    
    #Create figure and Axes. And set title.
    fig, axes = plt.subplots(2, 2, figsize=(10,6), gridspec_kw=dict(wspace=0.1, hspace=0.6))
    fig.suptitle(f"Defensive players' information with {style_available} style", fontsize=15)

    #Too check layout, I'll show text on each Axes.
    gs = axes[0, 1].get_gridspec()
    axes[0, 0].remove()
    axes[1, 0].remove()
    #Add gridspec we got
    axbig = fig.add_subplot(gs[:, 0])


    #Add three plots.
    sns.countplot(df_players["position"], ax=axbig)
    axbig.set_title("Number of players per position", fontsize=12)

    df_height.loc[:,["height","Count"]].set_index("height").plot(kind='bar', ax=axes[0, 1])
    axes[0, 1].set_ylabel("Count") 
    axes[0, 1].get_legend().remove()
    axes[0, 1].set_title('Player Height (ft-in)', fontsize=12)

    sns.distplot(df_players["weight"], kde=False, rug=False, ax=axes[1, 1])
    axes[1, 1].set_xlabel("Weight") 
    axes[1, 1].set_title('Player Weight (lbs)', fontsize=12)

In [ ]:
for style_available in plt.style.available:
    titanic_subplot(style_available)

We could easily change the look of the graphs.

Depending on the style, the texts color may have been assimilated to the background, so this needs to be corrected.

<a id="1"></a> <br>
# <div class="alert alert-block alert-info">Tip3: Color</div>

Using palette for seaborn and colormap for matplotlib, we can also change each plot's color. If you don't like the colors you've chosen by style, you can change them in this way.

In seaborn, we pass following strings or a list of colors specified any way that matplotlib accepts(an RGB tuple, a hex code, or a name in the X11 table) to **sns.color_palette()**, we can get colormaps. And we can pass it to **palette** augment. For pandas.DataFrame.plot, we can also pass the strings to **colormap** augment.

To know more detail, following tutrials will help you,

- [Choosing color palettes](https://seaborn.pydata.org/tutorial/color_palettes.html)

- [Choosing Colormaps in Matplotlib](https://matplotlib.org/tutorials/colors/colormaps.html)

In [ ]:
def titanic_subplot(style_available, palette_available):
    #set style
    plt.style.use(style_available)
    
    #get palette
    palette = sns.color_palette(palette_available)
    
    #Create figure and Axes. And set title.
    fig, axes = plt.subplots(2, 2, figsize=(10,6), gridspec_kw=dict(wspace=0.1, hspace=0.6))
    fig.suptitle("Defensive players' information", fontsize=15)

    #Too check layout, I'll show text on each Axes.
    gs = axes[0, 1].get_gridspec()
    axes[0, 0].remove()
    axes[1, 0].remove()
    #Add gridspec we got
    axbig = fig.add_subplot(gs[:, 0])


    #Add three plots.
    sns.countplot(df_players["position"], ax=axbig, palette=palette)
    axbig.set_title("Number of players per position", fontsize=12)

    df_height.loc[:,["height","Count"]].set_index("height").plot(colormap=palette_available, kind='bar', ax=axes[0, 1])
    axes[0, 1].set_ylabel("Count") 
    axes[0, 1].get_legend().remove()
    axes[0, 1].set_title('Player Height (ft-in)', fontsize=12)

    #Unfortunately, it does not seem to be possible to specify colormap.
    sns.distplot(df_players["weight"], kde=False, rug=False, ax=axes[1, 1])
    axes[1, 1].set_xlabel("Weight") 
    axes[1, 1].set_title('Player Weight (lbs)', fontsize=12)

In [ ]:
titanic_subplot("bmh", "Accent")

In [ ]:
titanic_subplot("bmh", "copper_r")

In [ ]:
titanic_subplot("dark_background", "plasma")

In [ ]:
titanic_subplot("dark_background", "YlOrRd")

The same style but with a different color of the figure made a big difference in the impression.

-------------

These were the tips I had gathered while create graphs and read good notebooks.

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#3cb371','#eb3446','Big Shoulders Stencil Text','Smokum',45,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Have a nice plot!' )